# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [66]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [70]:

df_raw  = pd.read_csv('Data/Cities_clean.csv')
google_url = 'base_url = "https://maps.googleapis.com/maps/api/geocode/json'

df_raw

,Unnamed: 0,index,city,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,0,lisala,2.1500,21.5167,70.16,95.0,100.0,2.26,1.627346e+09
1,1,1,gweta,-20.1833,25.2333,45.88,70.0,0.0,9.75,1.627346e+09
2,2,2,arrifes,37.7667,-25.7000,68.25,88.0,75.0,8.05,1.627346e+09
3,3,3,rizhao,35.4275,119.4553,80.55,88.0,99.0,18.75,1.627346e+09
4,4,4,palu,-0.8917,119.8707,84.33,70.0,100.0,1.05,1.627346e+09
...,...,...,...,...,...,...,...,...,...,...
558,563,610,bethel,41.3712,-73.4140,79.75,80.0,75.0,4.61,1.627347e+09
559,564,611,mataura,-46.1927,168.8643,51.48,78.0,22.0,4.47,1.627347e+09
560,565,612,albany,42.6001,-73.9662,77.04,79.0,0.0,5.82,1.627346e+09
561,566,613,saint-philippe,-21.3585,55.7679,63.99,83.0,67.0,11.18,1.627347e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [68]:
# configure the gmaps and putting int the google API key.
# gmaps.configure(api_key=g_key)

# isolate the locations that we want.\
# we want specificall the Lat, Long, and Humidity Data.
test_data  = df_raw[['Lat','Long','Humidity']].head()
test_data



In [69]:
# add the heatmaps according to the test data. 
# to do this, we need to create the heat layer and add it to google maps
# activate google maps 
fig = gmaps.figure()
locations = test_data[["Lat", "Long"]]
rating = test_data["Humidity"].astype(float)
max_humidity = test_data['Humidity'].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
# max temperature lower than 80 but higher than 70
df_temp = df_raw.loc[df_raw['Max Temp'] < 80 ]
df_temp = df_temp.loc[df_raw['Max Temp'] > 70 ]
# # wind speed less than 10mph
# df_wind = df_temp.loc[df_temp['Wind Speed'] < 10]
# # zero cloudiness
# df_cloud = df_wind.loc[df_wind['Cloudiness'] == 0]
# df_clean = df_cloud.reset_index(drop=True)
# df_clean
df_raw

,Unnamed: 0,index,city,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,0,lisala,2.1500,21.5167,70.16,95.0,100.0,2.26,1.627346e+09
1,1,1,gweta,-20.1833,25.2333,45.88,70.0,0.0,9.75,1.627346e+09
2,2,2,arrifes,37.7667,-25.7000,68.25,88.0,75.0,8.05,1.627346e+09
3,3,3,rizhao,35.4275,119.4553,80.55,88.0,99.0,18.75,1.627346e+09
4,4,4,palu,-0.8917,119.8707,84.33,70.0,100.0,1.05,1.627346e+09
...,...,...,...,...,...,...,...,...,...,...
558,563,610,bethel,41.3712,-73.4140,79.75,80.0,75.0,4.61,1.627347e+09
559,564,611,mataura,-46.1927,168.8643,51.48,78.0,22.0,4.47,1.627347e+09
560,565,612,albany,42.6001,-73.9662,77.04,79.0,0.0,5.82,1.627346e+09
561,566,613,saint-philippe,-21.3585,55.7679,63.99,83.0,67.0,11.18,1.627347e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
